In [ ]:
!pip install transformers
!pip install bert-extractive-summarizer

In [ ]:
!pip install spacy

In [ ]:
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import json

In [ ]:
with open('/content/drive/MyDrive/memoire/topicsdb.json', 'r') as fichier_json:
    rslt = json.load(fichier_json)

In [ ]:
!ls /content/drive/MyDrive/memoire/topicsdb.json

/content/drive/MyDrive/memoire/topicsdb.json


In [ ]:
from summarizer import Summarizer
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def resume_extraction(data):
  for i, topic in enumerate(data) : # Pour chaque sujet
    docs =  topic['article']   # on recupère le contenu articles associés
    topic_article = ""
    for doc in docs:
        topic_article = topic_article + " " + doc['contenuArticle'] # contenu de tous les articles associés

    model = Summarizer() # initialisation du modèle
    nb_sentences = model.calculate_optimal_k(topic_article[:1000000], k_max=10)  # determination du nombre de phrases optimales
    print(nb_sentences)
    resume = model(topic_article[:1000000], num_sentences=nb_sentences) # génération du résumé
    data[i]["resume"] = resume # ajout du résumé

In [ ]:
rslt = resume_extraction(data=rslt)

In [ ]:
rslt[0]['resume']

"On est loin d’en finir avec l’histoire de la grâce accordée à Khalifa Sall, élargi de prison après plus de 2 ans passés à la Maison d’arrêt et de correction de Rebeuss. Ce dernier accusé d’avoir écrit une lettre de grâce et adressée au Président de la République Macky Sall sans l’accord de l’ancien maire de Dakar ainsi que son codétenu, Yaya Bodian, va être traduit d’après «Les Echos» devant le Conseil de discipline de l’Ordre des avocats. « Des menaces qui ont eu le don de provoquer l'ire de la jeunesse Apériste de Touba qui a tenu à faire certaines mises au point."

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity
import spacy

In [ ]:
! python -m spacy download fr_core_news_sm

In [ ]:
nlp = spacy.load('fr_core_news_sm')

In [ ]:
def train_model(docs):
    # Création des documents tagués pour Doc2Vec
    tagged_data = [TaggedDocument(words=nlp(_d.lower()), tags=[str(i)]) for i, _d in enumerate(docs)]
    # Entraînement du modèle Doc2Vec
    doc2vec_model = Doc2Vec(vector_size=100, window=2, min_count=1, workers=4, epochs=200)
    doc2vec_model.build_vocab(tagged_data)
    doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

    return doc2vec_model

In [ ]:
def measure_similarity(doc1, doc2, model):
    token1 = [token.text.lower()  for token in nlp(doc1)]
    token2 = [token.text.lower()  for token in nlp(doc2)]

    # Obtention des vecteurs de chaque document
    vector1 = model.infer_vector(token1)
    vector2 = model.infer_vector(token2)

    # Calcul de la similarité cosinus
    similarity = cosine_similarity([vector1], [vector2])[0][0]

    return similarity

In [ ]:
def measure_articles_similarities(data):
  for i, topic in enumerate(data):
    docs = topic["article"] # Pour chaque sujet
    articles =  [doc['contenuArticle'] for doc in docs] # On repuère le contenu des articles associés
    articles.append(data[0]["resume"]) # Puis on insère le resumé de sujet en dernière position
    doc2vec_model = train_model(docs=articles) # Entrainons le modèle doc2Vec
    similarities = [measure_similarity(doc1=article, doc2=articles[-1], model=doc2vec_model) for article in articles[:-1]] #Pour chaque article de meme sujet on calcul sa similarité avec le resumé de sujet.
    data[i]['varianceSimilarite'] = np.var(np.array(similarities))
    data[i]['nombreArticle'] = len(similarities)
    for j, similarity in enumerate(similarities):
        data[i]["article"][j]["similarite"] = similarities[j]
  return data

In [ ]:
rslt = measure_articles_similarities(data=rslt)

In [ ]:
rslt[0]['varianceSimilarite']

0.009926182

In [ ]:
rslt[0]['nombreArticle']

172

In [ ]:
rslt[-2]['varianceSimilarite']

0.009539226

In [ ]:
rslt[-2]['nombreArticle']

96

In [ ]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [ ]:
with open("/content/drive/MyDrive/memoire/topicsdb-final.json", "w", encoding="utf-8") as data:
    json.dump(rslt, data, ensure_ascii=False, indent=4, cls=NpEncoder)